In [2]:
USER_FLAG = "--user"

In [3]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0
! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q

  Using cached google_cloud_aiplatform-1.7.0-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.16.1
    Uninstalling google-cloud-aiplatform-1.16.1:
      Successfully uninstalled google-cloud-aiplatform-1.16.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

In [2]:
import os
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  monstyrxai


In [3]:
BUCKET_NAME="gs://" + PROJECT_ID + "-bucket"

In [4]:
import kfp
import google.cloud.aiplatform as aip

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from typing import NamedTuple


In [5]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://monstyrxai-bucket/pipeline_root/'

In [17]:
import time
DISPLAY_NAME = 'monstyrxai-continuous-training{}'.format(str(int(time.time())))
print(DISPLAY_NAME)

monstyrxai-continuous-training1660109186


In [18]:
@kfp.dsl.pipeline(name="custom-classification-pipeline",
                  pipeline_root=PIPELINE_ROOT)
def pipeline(
    gcs_source : str = "gs://bucket-for-outputcsv/output.csv",
    display_name: str = DISPLAY_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    enable_caching: bool = False
    # thresholds_dict_str: str = '{"auRoc": 0.95}',
):
    
    text_dataset_create_op = gcc_aip.TextDatasetCreateOp(
        project=project, display_name="custom_classification_dataset", gcs_source=gcs_source, import_schema_uri=aip.schema.dataset.ioformat.text.multi_label_classification,
    )
    
    
    classification_training_op = gcc_aip.AutoMLTextTrainingJobRunOp(
        dataset=text_dataset_create_op.outputs["dataset"], display_name = display_name, multi_label = True, project = project, prediction_type = "classification"
    )
    
    
    
    endpoint_op = gcc_aip.EndpointCreateOp(
            project=project,
            location=gcp_region,
            display_name="classification-model",
        )
    
    gcc_aip.ModelDeployOp(
            model=classification_training_op.outputs["model"],
            endpoint=endpoint_op.outputs["endpoint"],
    )
    
    
    
    

In [19]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="compiled_pipeline.json"
)

In [20]:
!gsutil cp ./compiled_pipeline.json gs://monstyrxai-bucket/compiled_pipeline.json

Copying file://./compiled_pipeline.json [Content-Type=application/json]...
/ [1 files][ 15.0 KiB/ 15.0 KiB]                                                
Operation completed over 1 objects/15.0 KiB.                                     


In [20]:
aip.Endpoint.list(order_by="update_time desc")[0]

resource name: projects/276757795685/locations/us-central1/endpoints/6335641085942956032